In [ ]:
from google.colab import userdata
import requests
import json
together_apikey=userdata.get('togetherapi')
firework_apikey=userdata.get('fireapi')

In [ ]:
def togetherai(question,model,api_key=together_apikey):
    # url = "https://api.fireworks.ai/inference/v1/chat/completions" #for FW
    url = "https://api.together.xyz/v1/chat/completions"  #-- for Together AI
    formatted_prompt = f"Regenerate provided text: TEXT = {question}"
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": formatted_prompt}],
        "max_tokens": 1024,
        "top_p": 1,
        "top_k": 40,
        "presence_penalty": 0,
        "frequency_penalty": 0,
        "temperature": 0.5,
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        # Extract the output message
        output_message = response.json()['choices'][0]['message']['content']
        return output_message
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

In [ ]:
def fireworks(question,api_key=firework_apikey,model="accounts/yi-01-ai/models/yi-large"):
    url = "https://api.fireworks.ai/inference/v1/chat/completions" #for FW
    # url = "https://api.together.xyz/v1/chat/completions"  #-- for Together AI
    formatted_prompt = f"Regenerate the text: TEXT={question}\n"
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": formatted_prompt}],
        "max_tokens": 1024,
        "top_p": 1,
        "top_k": 40,
        "presence_penalty": 0,
        "frequency_penalty": 0,
        "temperature": 0.5,
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        # Extract the output message
        output_message = response.json()['choices'][0]['message']['content']
        return output_message
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None



In [ ]:
together_ai_models=["Qwen/Qwen2-72B-Instruct","google/gemma-2-9b-it","mistralai/Mistral-7B-Instruct-v0.1","meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"]



In [ ]:
class Score:
    def __init__(self,edit_score,new_text,model):
        self.edit_score=edit_score
        self.new_text=new_text
        self.model=model


In [ ]:
!pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.7 MB/s eta 0:00:00


In [ ]:
from nltk.tokenize import word_tokenize
import Levenshtein
import nltk
nltk.download('punkt_tab')
def get_edit_distance(text1, text2):
    tokens1 = word_tokenize(text1)
    tokens2 = word_tokenize(text2)
    joined1 = " ".join(tokens1)
    joined2 = " ".join(tokens2)
    distance = Levenshtein.distance(joined1, joined2)
    return distance


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
def detect_text(text):
    edit_distance_score=[]
    for model in together_ai_models:
        new_text=togetherai(text,model)
        edit_score=get_edit_distance(text,new_text)
        edit_distance_score.append(Score(edit_score,new_text,model))
    new_text=fireworks(text)
    edit_score=get_edit_distance(text,new_text)
    edit_distance_score.append(Score(edit_score,new_text,"Yi-Large"))
    return edit_distance_score

In [ ]:
text="""
Global Economy Global Economy Supported by World Bank Sees Rosier Growth Outlook But rising trade barriers pose a long-term threat to global output as protectionist policies spread, the bank said. ByAlan Rappeport Reporting from Washington The World Bank on Tuesday raised its outlook for the world economy this year but warned that the rise of new trade barriers and protectionist policies posed a long-term threat to global growth. In its latest Global Economic Prospects report, the World Bank projected global growth to hold steady at 2.6 percent this year, an upgrade from itsJanuary forecastof 2.4 percent, and predicted that output would edge higher to 2.7 percent in 2025. The forecasts showed the global economy stabilizing after being rocked in recent years by the pandemic and the wars in Ukraine and the Middle East. “Four years after the upheavals caused by the pandemic, conflicts, inflation and monetary tightening, it appears that global economic growth is steadying,” Indermit Gill, the World Bank’s chief economist, said in a statement accompanying the report. However, sluggish growth continues to haunt the world’s poorest economies, which are still grappling with inflation and the burdens of high debt. The bank noted that over the next three years, countries that account for more than 80 percent of the world’s population would experience slower growth than in the decade before the pandemic. The slightly brighter forecast was led by the resilience of the U.S. economy, which continues to defy expectations despite higher interest rates. Overall, advanced economies are growing at an annual rate of 1.5 percent, with output remaining sluggish in Europe and Japan. By contrast, emerging market and developing economies are growing at a rate of 4 percent, led by China and Indonesia. Although growth is expected to be a bit stronger than previously forecast, the World Bank said prices were easing more slowly than it projected six months ago. It foresees global inflation moderating to 3.5 percent in 2024 and 2.9 percent next year. That gradual decline is likely to lead central banks to delay interest rate cuts, dimming prospects for growth in developing economies."""

In [ ]:
data=detect_text(text)

In [ ]:
for item in data:
    print(f"{item.model} -- {item.edit_score}")

Qwen/Qwen2-72B-Instruct -- 1047
google/gemma-2-9b-it -- 1640
mistralai/Mistral-7B-Instruct-v0.1 -- 834
meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo -- 165
Yi-Large -- 1221
